# MNIST

아래의 code는 google colab 에서 시험된 결과이다.
Anaconda 에서 제공되는 jupyter notebook에서는 작동이 안될수 있다.
추천하는 방법은 notebook이 아니라 python 파일 자체를 실행하라는 것이다. TPU 예제만 빼고는 anaconda prompt 에서 *.py 파일을 실행하는 방법을 추천한다.
compare 도구로서는 "BCcompare"를 추천한다. working day로 30일간 무료로 사용이 가능하다.
무료 사용후 재 설치 하면 다시 30 working day동안 사용이 가능하다.

Compare 도구를 사용하여, "21_TF2_MNIST_expert_sequential_sparse.py"와 "23_TF2_MNIST_expert_subclass.py"를 비교하여 차이가 나는 부분을 살펴보자.

아래의 code 부분만 변경된것을 볼수 있다.

```
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(64, 3, activation='relu', padding='SAME')
        self.maxpool2d1 = MaxPool2D(padding='SAME')
        self.conv2 = Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME')
        self.maxpool2d2 = MaxPool2D(padding='SAME')
        self.conv3 = Conv2D(filters=256, kernel_size=3, activation=tf.nn.relu, padding='SAME')
        self.maxpool2d3 = MaxPool2D(padding='SAME')
        self.flatten = Flatten()
        self.d1 = Dense(256, activation='relu')
        self.d2 = Dropout(0.2)
        self.d3 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool2d1(x)
        x = self.conv2(x)
        x = self.maxpool2d2(x)
        x = self.conv3(x)
        x = self.maxpool2d3(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        return self.d3(x)

model = MyModel()

# model.summary()

```
Colab에서 제공하는 전문가를 위한 빠른 시작 부분에서는 갑자기 code가 바뀌었다. Colab 에서 제공하는 "초보자를 위한 빠른시작"과 "전문가를 위한 빠른 시작"의 큰 차이점은 두개임을 기억하자.
"tf.data.Dataset."와 "tf.GradientTape()"이 두가지가 큰 차이가 있다.

위의 network 정의하는 방법은 세가지 방법중 한가지 임을 기억하자.

아래부분은 "21_TF2_MNIST_expert_sequential_sparse.py"와 "23_TF2_MNIST_expert_subclass.py" 가 동일한 부분이다.

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras import Model, Sequential

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# print(tf.__version__)
## MNIST Dataset #########################################################
mnist = tf.keras.datasets.mnist
# class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
##########################################################################

## Fashion MNIST Dataset #################################################
# mnist = tf.keras.datasets.fashion_mnist
# class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
##########################################################################
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()    

# Change data type as float. If it is unt type, it might cause error 
X_train, X_test = X_train / 255.0, X_test / 255.0

print(Y_train[0:10])

print(X_train.shape)

# in the case of Keras or TF2, type shall be [image_size, image_size, 1]
# if it is RGB type, type shall be [image_size, image_size, 3]
# For MNIST or Fashion MNIST, it need to reshape
X_train = X_train[..., tf.newaxis]
X_test = X_test[..., tf.newaxis]

print(X_train.shape)

batch_size = 1000
# 입력된 buffer_size만큼 data를 채우고 무작위로 sampling하여 새로운 data로 바꿉니다.
# 완벽한 셔플링을 위해서는 데이터 세트의 전체 크기보다 크거나 같은 버퍼 크기가 필요합니다.
# 만약 작은 데이터수보다 작은 buffer_size를 사용할경우,
# 처음에 설정된 buffer_size만큼의 data안에서 임의의 셔플링이 발생합니다.
shuffle_size = 100000

train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, Y_train)).shuffle(shuffle_size).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(batch_size)


11493376/11490434 [==============================] - 0s 0us/step
[5 0 4 1 9 2 1 3 1 4]
(60000, 28, 28)
(60000, 28, 28, 1)


아래부분은 "21_TF2_MNIST_expert_sequential_sparse.py"와 "23_TF2_MNIST_expert_subclass.py" 는 network의 정의만 다르다..

In [2]:

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(64, 3, activation='relu', padding='SAME')
        self.maxpool2d1 = MaxPool2D(padding='SAME')
        self.conv2 = Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME')
        self.maxpool2d2 = MaxPool2D(padding='SAME')
        self.conv3 = Conv2D(filters=256, kernel_size=3, activation=tf.nn.relu, padding='SAME')
        self.maxpool2d3 = MaxPool2D(padding='SAME')
        self.flatten = Flatten()
        self.d1 = Dense(256, activation='relu')
        self.d2 = Dropout(0.2)
        self.d3 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool2d1(x)
        x = self.conv2(x)
        x = self.maxpool2d2(x)
        x = self.conv3(x)
        x = self.maxpool2d3(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        return self.d3(x)

model = MyModel()

# model.summary()


아래부분은 "21_TF2_MNIST_expert_sequential_sparse.py"와 "23_TF2_MNIST_expert_subclass.py" 가 동일한 부분이다.

In [3]:

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = 'epoch: {:>5,d}, loss: {:>2.4f}, accuracy: {:>2.3f}%, test loss: {:>2.4f}, test accuracy: {:>2.3f}%'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

epoch:     1, loss: 0.5038, accuracy: 84.645%, test loss: 0.0959, test accuracy: 96.890%
epoch:     2, loss: 0.2897, accuracy: 91.166%, test loss: 0.0701, test accuracy: 97.690%
epoch:     3, loss: 0.2087, accuracy: 93.631%, test loss: 0.0572, test accuracy: 98.090%
epoch:     4, loss: 0.1654, accuracy: 94.951%, test loss: 0.0504, test accuracy: 98.298%
epoch:     5, loss: 0.1378, accuracy: 95.795%, test loss: 0.0457, test accuracy: 98.462%
